# Initial data preparation

To download raw data from Princeton Server - see Readme.
This file will read and merge 1 year of data to pandas dataframe with no modifications, while keeping the eggs names and NANs for the parts where no data is available.

We will also remove "rotten eggs", a.k.a. eggs having more than 20% of year data unavailable, having mean yearly value out of 95% expectatition and final cumulative Z score probabily (under DF = seconds_in_year) out of 95% expectation. We know from original experiments that there are rotten eggs. Plus, we know that that there are eggs with non-neutral standart deviation, arising from the fact that XOR filtering method (link) is guaranteed to remove the bias, but not mean. 

As for the XOR method general - see sketches/XOR_understanding.ipynb to get impressions of how it works and what std bias it introduces.

In [74]:
import pandas as pd
import csv
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import math
from scipy.stats import chi2
import gzip
from joblib import Parallel, delayed
from tqdm.notebook import trange, tqdm
import glob

In [2]:
#The 200-bit trial sums have expected mean = 100 and standard deviation = 7.071 (sqrt(N)/2)


In [75]:
STD_200_FLIPS = math.sqrt(200)/2
N_JOBS = 8
YEAR = 2014

COVERAGE_THR = 0.2

In [76]:
def split_into_chunks(data, chunks=8):
    return [data[i:i + chunks] for i in range(0, len(data), chunks)]

def flatten(t):
    return [item for sublist in t for item in sublist]

In [78]:
filename = glob.glob('eggsummary/'+str(YEAR)+'/*')[0]
with gzip.open(filename, mode='rt') as f:
    reader = csv.reader(f)
    i=0
    for line in reader:
        i+=1
        if line[0]=='12': 
            egg_names = line[3:]
            break
data = pd.read_csv(filename, skiprows=i, names=['rowtype','timestamp','HRtimestamp']+egg_names)

In [79]:
# Create real data
def read_eggs_by_chunk(chunk, gen_synthetic_version=False):
    datas = []
    for filename in chunk:
        with gzip.open(filename, mode='rt') as f:
            reader = csv.reader(f)
            i=0
            for line in reader:
                i+=1
                if line[0]=='12': 
                    egg_names = line[3:]
                    break
        data = pd.read_csv(filename, skiprows=i, names=['rowtype','timestamp','HRtimestamp']+egg_names)

        if not gen_synthetic_version:
            datas.append(data)
            
        
        # this block allows to generate fully syntethic version of the dataset, that has NANs in the same places
        # as original data. Usefull sanity check - we are not to expect to have any deviations on this dataset.
        # If any are found - there's a bug and it should be smashed 
        else:
            syn_data = data.copy()
            random_set = np.random.default_rng().binomial(200,0.5,(data.shape[0],data.shape[1]-3))

            syn_data.iloc[:,3:] = random_set
            mask = data.iloc[:,3:].isna()   
            syn_data.iloc[:,3:] = syn_data.iloc[:,3:].where(~mask,other=np.nan)

            assert data.iloc[:,3:].isna().sum().sum() == syn_data.iloc[:,3:].isna().sum().sum()

            datas.append(syn_data)
            
    return datas


In [80]:
file_chunks = split_into_chunks(glob.glob('eggsummary/2014/*'), 8)
datas =  Parallel(n_jobs=N_JOBS)(delayed(read_eggs_by_chunk)(i) for i in tqdm(file_chunks))

  0%|          | 0/46 [00:00<?, ?it/s]

/home/anonymous/anaconda3/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


In [81]:
datas = flatten(datas)
data = datas[0].append(datas[1:])

In [82]:
del datas # save some RAM

In [98]:
# Looking for rotten eggs
BAD_EGGS = set()

In [99]:
for i in range(3,data.shape[1]):
    tmp_normalized = (data.iloc[:,i]-100)/STD_200_FLIPS
    tmp_no_na = tmp_normalized[~np.isnan(tmp_normalized)]
    
    missing_values = (data.iloc[:,i].isna().sum()/ data.shape[0])
    if missing_values>COVERAGE_THR:
        print('bad egg missing '+ data.columns[i])
        BAD_EGGS.add(data.columns[i])
        continue

    cumulative_sum = np.cumsum(tmp_normalized[~np.isnan(tmp_normalized)]).iloc[-1]
    probability_of_bin_dist = stats.binom_test((tmp_no_na>0).sum(), len(tmp_no_na)-(tmp_no_na==0).sum(), 0.5)

    if probability_of_bin_dist<0.05:
        print('bad egg '+ data.columns[i])
        BAD_EGGS.add(data.columns[i])
        continue
    
   

    print(i, ' ',data.columns[i], tmp_normalized.mean(), 
          missing_values,
          cumulative_sum,
          probability_of_bin_dist)
          

3   1 -0.00010486672695884537 0.09970535261288686 -2977.3438128636344 0.7106917069858326
4   37 4.983113295345877e-05 3.170979198376459e-07 1571.474110508946 0.4779616266867555
bad egg 103
6   108 -1.6947367335382404e-05 0.1262579908675799 -466.9733182956127 0.11699612406809227
7   110 -0.0003443595922612068 0.026902587519025876 -10567.56942347501 0.09590436198807503
8   111 0.00017445208353553923 0.005054477422628108 5473.7135931653675 0.5838044193791835
9   112 7.412333831955948e-05 0.0 2337.5535972456278 0.6400574375683606
10   116 0.0001815450113151531 0.006652175291730086 5687.118419725942 0.32512024236151993
11   118 0.00012706552383493164 0.05839992389649924 3773.1217844116104 0.9488868801330039
12   226 -0.00033318215093655206 0.10803012430238458 -9372.134699202725 0.19273663299961774
bad egg missing 228
14   231 -6.624460110985767e-05 0.0013603500761035009 -2086.2478472128587 0.17226736433407686
15   1021 8.748978061316539e-05 0.05564564307458143 2605.547067315796 0.3725509531

In [100]:
BAD_EGGS

{'1004',
 '103',
 '105',
 '1070',
 '1092',
 '1096',
 '1101',
 '1223',
 '1251',
 '2002',
 '2006',
 '2027',
 '2042',
 '2047',
 '2052',
 '2084',
 '2091',
 '2201',
 '2202',
 '2220',
 '2222',
 '2234',
 '224',
 '2241',
 '228',
 '3066',
 '3104',
 '3115',
 '4002',
 '4101'}

In [101]:

data.sort_values('timestamp',inplace=True)
data.drop(BAD_EGGS,axis=1,inplace=True) 

In [102]:
data.to_parquet('data'+str(YEAR)+'.parquet')

In [103]:
del data

## data created successfully

In [104]:
# here would be nice to do the same but with synthetic data - I haven't done it yet